<a href="https://colab.research.google.com/github/j95081456/EQ_Adaptation_VA/blob/main/Mahjong_sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [97]:
import random
from collections import Counter
import matplotlib.pyplot as plt
import time

# Set simulation mode to True to disable visualizations and delays.
SIMULATION = True

# Define Tile class
class Tile:
    def __init__(self, suit, value):
        self.suit = suit
        self.value = value

    def __repr__(self):
        return f"{self.suit}{self.value}"

    def __eq__(self, other):
        return self.suit == other.suit and self.value == other.value

    def __lt__(self, other):
        suit_order = {'B': 0, 'C': 1, 'D': 2, 'W': 3, 'G': 4}  # Bamboo, Characters, Dots, Winds, Dragons
        if self.suit == other.suit:
            return self.value < other.value
        return suit_order[self.suit] < suit_order[other.suit]

    def __hash__(self):
        return hash((self.suit, self.value))

# Define Player class
class Player:
    def __init__(self, name):
        self.name = name
        self.hand = []
        self.melded_sets = []  # Stores Chi, Peng, Gong sets

    def draw_tile(self, tile):
        self.hand.append(tile)
        self.hand.sort()

    def discard_tile(self, index=None):
        if index is not None and index < len(self.hand):
            return self.hand.pop(index)
        else:
            return self.hand.pop()

    def show_hand(self):
        return ' '.join(f"[{i}] {tile}" for i, tile in enumerate(self.hand))

    def can_peng(self, tile):
        return self.hand.count(tile) >= 2

    def can_chi(self, tile):
        if tile.suit not in ['B', 'C', 'D']:
            return False
        values = sorted([t.value for t in self.hand if t.suit == tile.suit])
        return ([tile.value - 1, tile.value + 1] in [values[i:i+2] for i in range(len(values) - 1)]) or \
               ([tile.value - 2, tile.value - 1] in [values[i:i+2] for i in range(len(values) - 1)]) or \
               ([tile.value + 1, tile.value + 2] in [values[i:i+2] for i in range(len(values) - 1)])

    def chi(self, tile):
        possible_sets = [[tile.value - 1, tile.value + 1],
                         [tile.value - 2, tile.value - 1],
                         [tile.value + 1, tile.value + 2]]
        values = sorted([t.value for t in self.hand if t.suit == tile.suit])
        for chi_set in possible_sets:
            if chi_set in [values[i:i+2] for i in range(len(values) - 1)]:
                chi_tiles = [Tile(tile.suit, chi_set[0]), tile, Tile(tile.suit, chi_set[1])]
                self.hand.remove(chi_tiles[0])
                self.hand.remove(chi_tiles[2])
                self.melded_sets.append(chi_tiles)
                print(f"{self.name} 吃 (Chi) {chi_tiles}")
                return True
        return False

    def peng(self, tile):
        self.hand.remove(tile)
        self.hand.remove(tile)
        self.melded_sets.append([tile, tile, tile])
        print(f"{self.name} 碰 (Peng) {tile}")
        return True

    def can_gong(self, tile, from_discard=False):
        # Check for upgrading an existing Peng meld first.
        for meld in self.melded_sets:
            if len(meld) == 3 and all(t == tile for t in meld):
                if self.hand.count(tile) >= 1:
                    return True
        if from_discard:
            # For a discarded tile, three copies in hand suffice.
            return self.hand.count(tile) >= 3
        else:
            # For a drawn tile, require four copies in hand.
            return self.hand.count(tile) >= 4

    def gong(self, tile):
        for meld in self.melded_sets:
            if len(meld) == 3 and all(t == tile for t in meld):
                if tile in self.hand:
                    self.hand.remove(tile)
                    meld.append(tile)
                    print(f"{self.name} 槓 (Gong) by upgrading Peng: {meld}")
                    return True
        if self.hand.count(tile) >= 4:
            for _ in range(4):
                self.hand.remove(tile)
            self.melded_sets.append([tile, tile, tile, tile])
            print(f"{self.name} 槓 (Gong) declared with {tile}")
            return True
        return False

    def check_hu(self):
        total_tiles = len(self.hand) + sum(3 if len(meld) == 4 else len(meld) for meld in self.melded_sets)
        if total_tiles != 14:
            return False
        tile_counts = Counter((tile.suit, tile.value) for tile in self.hand)
        pairs = [tile for tile, count in tile_counts.items() if count >= 2]
        for pair in pairs:
            temp_counts = tile_counts.copy()
            temp_counts[pair] -= 2  # Remove the pair
            if self._can_form_sets(temp_counts):
                return True
        return False

    def _can_form_sets(self, tile_counts):
        if sum(tile_counts.values()) == 0:
            return True
        for tile, count in tile_counts.items():
            if count >= 3:
                temp_counts = tile_counts.copy()
                temp_counts[tile] -= 3
                if self._can_form_sets(temp_counts):
                    return True
            suit, value = tile
            if suit in ['B', 'C', 'D']:
                if all((suit, value + i) in tile_counts and tile_counts[(suit, value + i)] > 0 for i in range(3)):
                    temp_counts = tile_counts.copy()
                    for i in range(3):
                        temp_counts[(suit, value + i)] -= 1
                    if self._can_form_sets(temp_counts):
                        return True
        return False

# Visualization functions (skipped during simulation)
def visualize_hand(player):
    if SIMULATION:
        return
    suits = {'B': 'Bamboo', 'C': 'Characters', 'D': 'Dots', 'W': 'Winds', 'G': 'Dragons'}
    suit_colors = {'B': 'green', 'C': 'red', 'D': 'blue', 'W': 'purple', 'G': 'gold'}
    fig, ax = plt.subplots(figsize=(8, 1.5))
    for idx, tile in enumerate(player.hand):
        ax.text(idx, 0.5, f'{tile.suit}{tile.value}', fontsize=10, ha='center', va='center',
                bbox=dict(facecolor=suit_colors[tile.suit], alpha=0.3))
    ax.set_xlim(-1, len(player.hand))
    ax.set_ylim(0, 1)
    ax.axis('off')
    plt.title(f"{player.name}'s Hand")
    plt.show()

def visualize_melds(player):
    if SIMULATION:
        return
    if not player.melded_sets:
        print(f"{player.name} has no melded sets.")
        return
    suit_colors = {'B': 'green', 'C': 'red', 'D': 'blue', 'W': 'purple', 'G': 'gold'}
    x_offset = 0
    fig, ax = plt.subplots(figsize=(len(player.melded_sets) * 2, 2))
    ax.axis('off')
    for meld in player.melded_sets:
        for idx, tile in enumerate(meld):
            ax.text(x_offset + idx, 0.5, f'{tile.suit}{tile.value}', fontsize=12, ha='center', va='center',
                    bbox=dict(facecolor=suit_colors[tile.suit], alpha=0.3))
        x_offset += len(meld) + 1
    plt.title(f"{player.name}'s Melded Sets (Chi, Peng, Gong)")
    plt.xlim(-1, x_offset)
    plt.ylim(0, 1)
    plt.show()

def ai_discard_strategy_1(player):
    # Smarter AI discard logic: Prefer discarding isolated tiles
    tile_counts = Counter((tile.suit, tile.value) for tile in player.hand)

    # Prioritize discarding unique, non-set-forming tiles
    for tile in player.hand:
        if tile_counts[(tile.suit, tile.value)] == 1:
            return tile

    # If no unique tile, discard the last tile
    return player.hand[-1]

def ai_discard_strategy_2(player):
    tile_scores = {}
    for tile in player.hand:
        score = 0
        count = player.hand.count(tile)

        # Duplicate bonus: more duplicates make the tile more valuable.
        # For each extra copy, reduce the score.
        score -= (count - 1) * 3

        if tile.suit in ['B', 'C', 'D']:
            # For suited tiles, check potential chow formations.
            chow_bonus = 0
            # Possible chow patterns including the tile:
            # 1. (tile, tile+1, tile+2)
            # 2. (tile-1, tile, tile+1)
            # 3. (tile-2, tile-1, tile)
            patterns = [
                (tile.value, tile.value + 1, tile.value + 2),
                (tile.value - 1, tile.value, tile.value + 1),
                (tile.value - 2, tile.value - 1, tile.value)
            ]
            for pattern in patterns:
                bonus = 0
                # The current tile is already in hand; check for the other two.
                for val in pattern:
                    if val == tile.value:
                        continue
                    if 1 <= val <= 9 and Tile(tile.suit, val) in player.hand:
                        bonus += 2  # Each neighboring tile contributes a bonus.
                chow_bonus = max(chow_bonus, bonus)
            score -= chow_bonus

            # If the tile is unique and has no immediate neighbors, add a heavy penalty.
            if count == 1:
                neighbors = [Tile(tile.suit, tile.value - 1), Tile(tile.suit, tile.value + 1)]
                if not any(neighbor in player.hand for neighbor in neighbors):
                    score += 5

        else:
            # For honor tiles (winds/dragons) no chow potential exists.
            if count == 1:
                score += 5

        tile_scores[tile] = score

    # The tile with the highest score (least potential) is chosen for discard.
    discard_tile = max(player.hand, key=lambda t: tile_scores[t])
    return discard_tile

def ai_discard_strategy_3(player):
    """
    Improved discard strategy:
    - For each tile, compute a 'potential' score that rewards duplicates,
      rewards the existence of immediate neighbors (which indicate chow potential),
      and rewards complete or nearly complete sequences.
    - The tile with the highest (i.e. worst) score (lowest potential) is chosen for discard.
    """
    potentials = {}
    for tile in player.hand:
        # Start with a base potential.
        potential = 0
        count = player.hand.count(tile)

        # Duplicates are very valuable.
        if count > 1:
            potential -= (count - 1) * 4  # bonus for each extra copy
        else:
            potential += 10  # isolated tile penalty

        if tile.suit in ['B', 'C', 'D']:
            # Check immediate neighbors (chow potential)
            neighbor_bonus = 0
            for offset in [-2, -1, 1, 2]:
                neighbor_val = tile.value + offset
                if 1 <= neighbor_val <= 9:
                    neighbor_tile = Tile(tile.suit, neighbor_val)
                    if neighbor_tile in player.hand:
                        neighbor_bonus += 3
            potential -= neighbor_bonus

            # Extra bonus if the tile is part of a nearly complete sequence.
            # For example, if tile, tile+1 exist (or tile-1, tile) then waiting for tile+2 (or tile-2) is possible.
            if Tile(tile.suit, tile.value - 1) in player.hand:
                potential -= 5
            if Tile(tile.suit, tile.value + 1) in player.hand:
                potential -= 5
        else:
            # Honor tiles: only duplicates count.
            if count == 1:
                potential += 5
            else:
                potential -= (count - 1) * 3

        potentials[tile] = potential

    # For debugging, you might print out each tile's potential:
    # for t, score in potentials.items():
    #     print(f"{t}: {score}")

    # Choose the tile with the highest potential value (i.e. the worst candidate to keep).
    discard_tile = max(player.hand, key=lambda t: potentials[t])
    return discard_tile


# Improved AI discard strategy
def ai_discard_strategy(player):
    tile_scores = {}
    for tile in player.hand:
        score = 0
        count = player.hand.count(tile)
        if count == 1:
            score += 3
        else:
            score -= count
        if tile.suit in ['B', 'C', 'D']:
            adjacent_bonus = 0
            if Tile(tile.suit, tile.value - 1) in player.hand:
                adjacent_bonus += 2
            if Tile(tile.suit, tile.value + 1) in player.hand:
                adjacent_bonus += 2
            if Tile(tile.suit, tile.value - 2) in player.hand:
                adjacent_bonus += 1
            if Tile(tile.suit, tile.value + 2) in player.hand:
                adjacent_bonus += 1
            score -= adjacent_bonus
        tile_scores[tile] = score
    discard_tile = max(player.hand, key=lambda t: tile_scores[t])
    return discard_tile

# Define Game class
class Game:
    def __init__(self, first_player):
        self.tiles = self.initialize_tiles()
        random.shuffle(self.tiles)
        self.players = [Player("NPC 1"), Player("NPC 2"), Player("NPC 3"), Player("NPC 4")]
        self.last_player = None  # Last player who discarded a tile
        self.current_player = first_player  # Index of current player
        self.distribute_tiles()

    def initialize_tiles(self):
        suits = ['B', 'C', 'D']
        winds = ['E', 'S', 'W', 'N']
        dragons = ['R', 'G', 'H']
        tiles = [Tile(suit, value) for suit in suits for value in range(1, 10) for _ in range(4)]
        tiles += [Tile('W', wind) for wind in winds for _ in range(4)]
        tiles += [Tile('G', dragon) for dragon in dragons for _ in range(4)]
        return tiles

    def distribute_tiles(self):
        for _ in range(13):
            for player in self.players:
                player.draw_tile(self.tiles.pop())

    def play_game(self):
        round_number = 1
        last_discarded_tile = None
        while True:
            tile_claimed = False
            player = self.players[self.current_player]

            # Check if any player can win by claiming the discarded tile.
            if last_discarded_tile:
                for p in self.players:
                    if p != self.last_player:
                        total_tiles = len(p.hand) + sum(3 if len(meld) == 4 else len(meld) for meld in p.melded_sets)
                        if total_tiles + 1 == 14:
                            p.hand.append(last_discarded_tile)
                            if p.check_hu():
                                print(f"🎉 {p.name} wins with a Hu hand by claiming the discarded tile {last_discarded_tile}! 🎉")
                                return p.name
                            else:
                                p.hand.pop()

            # Check for Gong, Peng, or Chi with the discarded tile.
            if last_discarded_tile:
                for p in self.players:
                    if p != self.last_player and p.can_gong(last_discarded_tile, from_discard=True):
                        tile_claimed = p.gong(last_discarded_tile)
                        if tile_claimed:
                            if self.tiles:
                                supplemental_tile = self.tiles.pop()
                                p.draw_tile(supplemental_tile)
                                print(f"{p.name} draws supplemental tile: {supplemental_tile}")
                            self.current_player = self.players.index(p)
                            player = p
                            last_discarded_tile = None
                            break

                if not tile_claimed:
                    for p in self.players:
                        if p != self.last_player and p.can_peng(last_discarded_tile):
                            tile_claimed = p.peng(last_discarded_tile)
                            if tile_claimed:
                                self.current_player = self.players.index(p)
                                player = p
                                last_discarded_tile = None
                                break

                if not tile_claimed and player.can_chi(last_discarded_tile):
                    tile_claimed = player.chi(last_discarded_tile)
                    if tile_claimed:
                        last_discarded_tile = None

            if tile_claimed and player.check_hu():
                print(f"🎉 {player.name} wins with a Hu hand! 🎉")
                return player.name

            print(f"\nRound {round_number} - {player.name}'s turn")
            # (Visualizations skipped in simulation mode.)
            visualize_hand(player)
            visualize_melds(player)

            # Draw a tile if available.
            if not tile_claimed and self.tiles:
                drawn_tile = self.tiles.pop()
                player.draw_tile(drawn_tile)
                print(f"{player.name} drew: {drawn_tile}")
                for tile in set(player.hand):
                    if player.can_gong(tile, from_discard=False):
                        if player.gong(tile):
                            if self.tiles:
                                supplemental_tile = self.tiles.pop()
                                player.draw_tile(supplemental_tile)
                                print(f"{player.name} draws supplemental tile: {supplemental_tile}")
                            break
                if player.check_hu():
                    print(f"🎉 {player.name} wins with a Hu hand! 🎉")
                    return player.name
            elif not self.tiles:
                print("No more tiles left! It's a draw.")
                return "Draw"

            # Discard a tile automatically.
            if player.name == "NPC 1":
                discard = ai_discard_strategy_3(player)
            else:
                discard = ai_discard_strategy(player)

           # discard = ai_discard_strategy(player)
            player.hand.remove(discard)
            last_discarded_tile = discard
            print(f"{player.name} discarded: {discard}")

            self.last_player = player
            self.current_player = (self.current_player + 1) % 4
            if self.current_player == 0:
                round_number += 1

           # if not SIMULATION:
           #     time.sleep(3)
           # else:
           #     time.sleep(0.1)

# Main simulation: Run 10 games and record wins.
if __name__ == "__main__":
    win_counts = {"NPC 1": 0, "NPC 2": 0, "NPC 3": 0, "NPC 4": 0, "Draw": 0}
    num_games = 1000
    for i in range(num_games):
        print(f"\n=== Starting Game {i+1} ===")
        game = Game(first_player = i%4)
        result = game.play_game()
        win_counts[result] += 1
        print(f"Game {i+1} result: {result}")
    print("\n=== Simulation Results ===")
    for player, wins in win_counts.items():
        print(f"{player}: {wins} win(s)")


串流輸出內容已截斷至最後 5000 行。

Round 5 - NPC 3's turn
NPC 3 discarded: C4

Round 5 - NPC 4's turn
NPC 4 drew: WN
NPC 4 discarded: WN
NPC 2 碰 (Peng) WN

Round 6 - NPC 2's turn
NPC 2 discarded: D9

Round 6 - NPC 3's turn
NPC 3 drew: C6
NPC 3 discarded: C6

Round 6 - NPC 4's turn
NPC 4 drew: WW
NPC 4 discarded: WW

Round 7 - NPC 1's turn
NPC 1 drew: D3
NPC 1 discarded: C2

Round 7 - NPC 2's turn
NPC 2 drew: B1
NPC 2 discarded: B1

Round 7 - NPC 3's turn
NPC 3 drew: C1
NPC 3 discarded: C1

Round 7 - NPC 4's turn
NPC 4 drew: D7
NPC 4 discarded: D7
NPC 3 碰 (Peng) D7

Round 8 - NPC 3's turn
NPC 3 discarded: D1

Round 8 - NPC 4's turn
NPC 4 drew: C9
NPC 4 discarded: C9

Round 9 - NPC 1's turn
NPC 1 drew: WW
NPC 1 discarded: WW

Round 9 - NPC 2's turn
NPC 2 drew: GG
NPC 2 discarded: WW

Round 9 - NPC 3's turn
NPC 3 drew: D5
NPC 3 discarded: GG

Round 9 - NPC 4's turn
NPC 4 drew: C3
NPC 4 discarded: C3

Round 10 - NPC 1's turn
NPC 1 drew: C5
NPC 1 discarded: B4

Round 10 - NPC 2's turn
NPC 2 drew: C7
NPC